#### Setup Param

In [ ]:
//  pass the parameters
dbutils.widgets.text("source_catalog", "ag_content_ims_acs")
dbutils.widgets.text("source_environment", "prod")
dbutils.widgets.text("source_version", "")

dbutils.widgets.text("target_catalog", "ag_ra_search_analytics_data")
dbutils.widgets.text("target_environment", "dev")
dbutils.widgets.text("target_version", "v1_0")
dbutils.widgets.text("pipeline_name", "")


dbutils.widgets.text("metadata_table_prefix", "")

// dynamic paramters
val source_catalog = dbutils.widgets.get("source_catalog")
val source_environment = dbutils.widgets.get("source_environment")
val source_version = dbutils.widgets.get("source_version")
val target_catalog = dbutils.widgets.get("target_catalog")
val target_environment = dbutils.widgets.get("target_environment")
val target_version = dbutils.widgets.get("target_version")
val pipeline_name = dbutils.widgets.get("pipeline_name")
val metadata_table_prefix = dbutils.widgets.get("metadata_table_prefix")



#### SchemaResolver

In [ ]:

object SchemaResolver {

  private def getWidget(name: String, default: String): String = {
    try {
      val value = dbutils.widgets.get(name)
      if (value == null || value.isEmpty) default else value
    } catch {
      case _: Throwable => default
    }
  }

  private val collection = getWidget("collection", "wos")
  private val source_catalog = getWidget("source_catalog_name", "ag_content_ims_acs")
  private val source_environment = getWidget("source_environment", "prod")
  private val source_version = getWidget("source_schema_version", "")
  private val source_table_prefix = getWidget("source_schema_prefix", "gold")

  private val target_catalog = getWidget("target_catalog_name", "ag_ra_search_analytics_data")
  private val target_environment = getWidget("target_environment", "dev")
  private val target_version = getWidget("target_schema_version", "v1_0")

  private val metadata_table_prefix = getWidget("metadata_table_prefix", "")
  private val dap_volume_schema = getWidget("dap_volume_schema", "")

  // define constant values initialized only once atstartup
  val DAP_VOLUME = s"/Volumes/${target_catalog}_${target_environment}/${dap_volume_schema}/reference/"

  val PIPELINE = getWidget("pipeline_name", "")
  // val OPS_SCHEMA = s"${target_catalog}_${target_environment}.dap_ops_${target_version}"
  val OPS_SCHEMA = s"${target_catalog}_${target_environment}.dap_ops_${target_version}"
  val OPS_TABLE_PREFIX: String = Option(metadata_table_prefix)
    .filter(_.trim.nonEmpty)
    .map(p => s"${p}_")
    .getOrElse("")



  private val dapSchemaBases = Seq(
      "dap_entity_wos",
      "dap_metrics_wos",
      "dap_entity_pprn",
      "dap_metrics_pprn",
      "dap_docs",
      "dap_reference",
      "dap_sort_ref",
      "dap_entity_enrich",
      "dap_grant",
      "dap_prod_core",
      "dap_ops",
      "dap_work"
  )

  private val acsSchemaBases= Seq(
      s"${source_table_prefix}_entity",
      s"${source_table_prefix}_wos",
      s"${source_table_prefix}_pprn"
  )

  // This map is used by QueryProcessor
  val SCHEMA_MAP: Map[String, String] = {
    val srcBase = s"${source_catalog}_${source_environment}"
    val tgtBase = s"${target_catalog}_${target_environment}"
    val versionSuffix = if (source_version.isEmpty) "" else s"_${source_version}"
    val tgtVerSuffix = s"_${target_version}"

    // ACS schemas: key → base name (without version)
    
    val acsSchemas = Seq(
     "entity" -> s"${srcBase}.${source_table_prefix}_entity${versionSuffix}",
     "wos"    -> s"${srcBase}.${source_table_prefix}_wos${versionSuffix}",
     "pprn"   -> s"${srcBase}.${source_table_prefix}_pprn${versionSuffix}"
    ) 

    // DAP schemas: key → base name (without target version suffix)
    val dapSchemas: Seq[(String, String)] =
      dapSchemaBases.map { key =>
        key -> s"$tgtBase.$key$tgtVerSuffix"
      }

    // Optional sandbox entry - for testing
    val sandboxEntry = Seq(
      "dap" -> s"$tgtBase.sandbox$tgtVerSuffix"
    )
    // Combine all
    (acsSchemas ++ dapSchemas ++ sandboxEntry).toMap
  }


  val DAP_SCHEMAS: Seq[String] = {

    val base = s"${target_catalog}_${target_environment}"
    dapSchemaBases.map { key =>
      s"$base.${key}_$target_version"
    }
  }

  val ACS_SCHEMAS: Seq[String] = {

    val version = if(source_version.isEmpty) "" else  s"_${source_version}"
    val base = s"${source_catalog}_${source_environment}"

    acsSchemaBases.map { key =>
      s"${base}.${key}${version}"
    }
  }

  val dapTablesForDeltaVersions = Seq(
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.${collection}",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.authorprofile",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.ap_article",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.organization",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.org_article",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.dept_article",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.journal",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.jcr_metrics",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.region",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.region_article_metrics",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.funding_agency",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.funding_agency_article_metrics",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.jcr_metrics_more",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.category",
    s"${target_catalog}_${target_environment}.dap_entity_${collection}_${target_version}.category_article_metrics",
    s"${target_catalog}_${target_environment}.dap_metrics_${collection}_${target_version}.article_normalized_metrics_woscore",
    s"${target_catalog}_${target_environment}.dap_metrics_${collection}_${target_version}.article_normalized_metrics",
    s"${target_catalog}_${target_environment}.dap_metrics_${collection}_${target_version}.category_metrics",
    s"${target_catalog}_${target_environment}.dap_metrics_${collection}_${target_version}.en_societal_facet",
    s"${target_catalog}_${target_environment}.dap_metrics_${collection}_${target_version}.en_societal_impact",
    s"${target_catalog}_${target_environment}.dap_metrics_${collection}_${target_version}.en_research_topics",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.incites_ri_organizations",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.incites_ri_funding_organizations",
    s"${target_catalog}_${target_environment}.dap_metrics_${target_version}.patent_org",
    s"${target_catalog}_${target_environment}.dap_metrics_${target_version}.patent_granted_application",
    s"${target_catalog}_${target_environment}.dap_prod_core_${target_version}.wos",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_grant_ut_funder",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_grant_ut_org",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_grants_article",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_grants_items",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_grant_funder",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_grant_org",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_authorprofile",
    s"${target_catalog}_${target_environment}.dap_grant_${target_version}.grants_ri_grants_uts" 
  )

}



#### Test

In [ ]:


SchemaResolver.ACS_SCHEMA_MAP.foreach{ case (k,v) => println(s"$k -> $v") }
println("------")
SchemaResolver.DAP_SCHEMAS.foreach{ schema => println(schema) }  
println("------")
SchemaResolver.ACS_SCHEMAS.foreach{ schema => println(schema) }  
println("------")

println(SchemaResolver.PIPELINE)